In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('traindatahack - train_again.csv')
df.head()

,city_name,activity_name,activity_description,activity_location,latitude,longitude,google_rating,category,families,couples,...,child-friendly,21+,cost,ada_accessibility,interactivity,atmosphere,estimated_duration,reservation_needed,rare_find,person_responses
0,Seattle,Blue Moon Tavern,A historic bar known for its eclectic atmosphe...,"712 NE 45th St, Seattle, WA 98105",47.6614,-122.3190,4.4,Drinks,0,1,...,0,1,Medium,1,Self-guided,Bustling,2.0,0,1,0
1,Boston,Boston Brewery Tours,Explore Boston's thriving craft beer scene wit...,Various Locations,42.3601,-71.0589,4.7,Drinks,0,0,...,0,1,Medium,1,Guided,Exciting,3.0,1,1,0
2,Dallas,Cidercade Dallas,Arcade with over 140 games and 24 ciders on tap,2777 Irving Blvd.,32.8000,-96.8475,4.8,Drinks,1,1,...,0,1,Low,1,Hands-on,Exciting,2.5,0,1,1
3,Dallas,Deep Ellum Brewing Co.,Brewery offering tours and tastings of craft b...,2823 St Louis St.,32.7829,-96.7833,4.5,Drinks,0,0,...,0,1,Medium,0,Guided,Bustling,1.5,0,0,0
4,Boston,Isabella’s,A cozy speakeasy bar with craft cocktails and ...,"1 Franklin St, Boston, MA 02110",42.3556,-71.0603,4.5,Drinks,0,1,...,0,1,Medium,1,Self-guided,Relaxing,1.5,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   city_name             330 non-null    object 
 1   activity_name         330 non-null    object 
 2   activity_description  330 non-null    object 
 3   activity_location     330 non-null    object 
 4   latitude              329 non-null    float64
 5   longitude             329 non-null    float64
 6   google_rating         330 non-null    float64
 7   category              330 non-null    object 
 8   families              330 non-null    int64  
 9   couples               330 non-null    int64  
 10  solo                  330 non-null    int64  
 11  child-friendly        330 non-null    int64  
 12  21+                   330 non-null    int64  
 13  cost                  330 non-null    object 
 14  ada_accessibility     330 non-null    object 
 15  interactivity         3

In [ ]:
from sklearn.model_selection import train_test_split
columns_to_keep = [ 'category', 'families', 'couples', 'solo', 'child-friendly', '21+', 'cost', 'ada_accessibility','interactivity', 'atmosphere', 'estimated_duration', 'reservation_needed', 'rare_find', 'person_responses']
full_data_set = df[columns_to_keep]
train_set, test_set = train_test_split(full_data_set, test_size=0.2, random_state=42)
print(train_set.shape)
print(test_set.shape)

(264, 14)
(66, 14)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
locations_train = train_set.drop("person_responses", axis=1)
train_labels = train_set["person_responses"].copy()
locations_test = test_set.drop("person_responses", axis=1)
test_labels = test_set["person_responses"].copy()

In [ ]:
data_cat = full_data_set.select_dtypes(include='object')

In [ ]:
cat_pipeline = Pipeline([
        ('imputer2', SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(sparse_output=False,  handle_unknown="ignore")),
    ])

cat_attribs = list(data_cat)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

full_pipeline = ColumnTransformer([
        ("cat", cat_pipeline, cat_attribs),
    ])

train_prepared = full_pipeline.fit_transform(locations_train)
test_prepared = full_pipeline.transform(test_set)

In [ ]:
print(full_data_set.shape)
print(train_prepared.shape)
train_prepared

(330, 14)
(264, 30)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Naive Bayes Model
# https://www.geeksforgeeks.org/multinomial-naive-bayes/

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

model = MultinomialNB()
model.fit(train_prepared, train_labels)

y_pred = model.predict(test_prepared)
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%\n")

Accuracy: 78.79%



In [ ]:
# SVM Model
# https://www.geeksforgeeks.org/support-vector-machine-algorithm/

import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svm = SVC(kernel="poly", gamma=10, C=1)
svm.fit(train_prepared, train_labels)

classifier_predictions = svm.predict(test_prepared)
print(accuracy_score(test_labels, classifier_predictions)*100)
print(classification_report(test_labels, classifier_predictions))

71.21212121212122
              precision    recall  f1-score   support

           0       0.47      0.44      0.46        18
           1       0.80      0.81      0.80        48

    accuracy                           0.71        66
   macro avg       0.63      0.63      0.63        66
weighted avg       0.71      0.71      0.71        66



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 5, 6, 7, 8, 9, 10, 100, 1000],
              'gamma': [1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(train_prepared, train_labels)
print(grid.best_params_)

grid_predictions = grid.predict(test_prepared)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.717 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.717 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.717 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.712 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.717 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.717 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.717 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.698 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.712 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.698 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=po

In [ ]:
# Logistic Regression Model
# https://www.geeksforgeeks.org/understanding-logistic-regression/

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(max_iter=10000, random_state=0)
clf.fit(train_prepared, train_labels)

acc = accuracy_score(test_labels, clf.predict(test_prepared)) * 100
print(f"Logistic Regression model accuracy: {acc:.2f}%")

Logistic Regression model accuracy: 78.79%


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, solver='saga', tol=0.1))

param_grid = {
    'logisticregression__C': [0.1, 1, 10, 100],
    'logisticregression__penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=5)

grid_search.fit(train_prepared, train_labels)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'logisticregression__C': 0.1, 'logisticregression__penalty': 'l2'}
Best Score: 0.7159651669085632


In [ ]:
# Decision Tree Model
# https://www.geeksforgeeks.org/building-and-implementing-decision-tree-classifiers-with-scikit-learn-a-comprehensive-guide/

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

tree_reg = DecisionTreeClassifier(random_state=41)
tree_reg.fit(train_prepared, train_labels)

y_pred = tree_reg.predict(test_prepared)

accuracy = accuracy_score(test_labels, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7272727272727273


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'max_depth': range(1, 10, 1),
    'min_samples_leaf': range(1, 20, 2),
    'min_samples_split': range(2, 20, 2),
    'criterion': ["entropy", "gini"]
}

tree = DecisionTreeClassifier(random_state=1)
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid,
                           cv=5, verbose=True)
grid_search.fit(train_prepared, train_labels)

print("best accuracy", grid_search.best_score_)
print(grid_search.best_estimator_)


Fitting 5 folds for each of 1620 candidates, totalling 8100 fits
best accuracy 0.7272133526850508
DecisionTreeClassifier(max_depth=3, random_state=1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation="relu", input_shape=(train_prepared.shape[1],)),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train Model
model.fit(train_prepared, train_labels, epochs=50, batch_size=32, validation_data=(test_prepared, test_labels))

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6522 - loss: 0.6659 - val_accuracy: 0.7273 - val_loss: 0.6259
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6867 - loss: 0.6187 - val_accuracy: 0.7273 - val_loss: 0.5855
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7174 - loss: 0.6030 - val_accuracy: 0.7273 - val_loss: 0.5676
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7305 - loss: 0.5765 - val_accuracy: 0.7273 - val_loss: 0.5615
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6995 - loss: 0.6164 - val_accuracy: 0.7273 - val_loss: 0.5660
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7010 - loss: 0.5920 - val_accuracy: 0.7273 - val_loss: 0.5621
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7283 - loss: 0.5486 - val_accuracy: 0.7273 - val_loss: 0.5508
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7124 - loss: 0.5798 - val_accuracy: 0.7273 - val_loss: 0.5452
Epoch 9/50


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_train = scaler.fit_transform(train_prepared)
X_test = scaler.transform(test_prepared)

model = MLPClassifier(hidden_layer_sizes=(32, 16), activation="relu", solver="adam", max_iter=500, random_state=41)
model.fit(train_prepared, train_labels)

y_pred = model.predict(test_prepared)

accuracy = accuracy_score(test_labels, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.68


In [ ]:
param_grid = {
    "hidden_layer_sizes": [(32,), (32, 16), (64, 32), (128, 64, 32)],  # Different layer structures
    "activation": ["relu", "tanh"],  # Activation functions
    "solver": ["adam"],  # Only using Adam optimizer
    "alpha": [0.0001, 0.001, 0.01],  # L2 Regularization
    "learning_rate": ["constant", "adaptive"],  # Learning rate strategies
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=2)
grid_search.fit(train_prepared, train_labels)

print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

y_pred = best_model.predict(test_prepared)
accuracy = accuracy_score(test_labels, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (32, 16), 'learning_rate': 'constant', 'solver': 'adam'}
Test Accuracy: 0.68


In [ ]:
# Adding RandomizedSearchCV for more fine-tuned modeling

mlp = MLPClassifier(max_iter=1000, early_stopping=True, random_state=41)

param_dist = {
    "hidden_layer_sizes": [(32,), (32, 16), (64, 32), (128, 64, 32)],
    "activation": ["relu", "tanh"],
    "solver": ["adam"],
    "alpha": [0.0001, 0.001, 0.01],
    "learning_rate": ["constant", "adaptive"],
    "batch_size": [16, 32]
}

random_search = RandomizedSearchCV(mlp, param_distributions=param_dist,
                                   n_iter=10, cv=3, scoring="roc_auc",
                                   n_jobs=-1, verbose=2, random_state=42)

random_search.fit(train_prepared, train_labels)

print(f"Best Parameters: {random_search.best_params_}")
best_model = random_search.best_estimator_

y_pred = best_model.predict(test_prepared)
accuracy = accuracy_score(test_labels, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (64, 32), 'batch_size': 16, 'alpha': 0.001, 'activation': 'tanh'}
Test Accuracy: 0.73


In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

scaler = StandardScaler()
X_train = scaler.fit_transform(train_prepared)
X_test = scaler.transform(test_prepared)

model = Sequential([
    Dense(64, activation="relu", kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)

history = model.fit(X_train, train_labels, epochs=100, batch_size=32, validation_data=(X_test, test_labels), callbacks=[early_stopping, lr_scheduler])

test_loss, test_acc = model.evaluate(X_test, test_labels)
print(f"\n Test Accuracy: {test_acc:.2f}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.6735 - loss: 1.3546 - val_accuracy: 0.7273 - val_loss: 0.9827 - learning_rate: 0.0100
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6755 - loss: 0.9941 - val_accuracy: 0.7273 - val_loss: 0.7581 - learning_rate: 0.0100
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7336 - loss: 0.7657 - val_accuracy: 0.7576 - val_loss: 0.6480 - learning_rate: 0.0100
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7299 - loss: 0.6372 - val_accuracy: 0.7879 - val_loss: 0.6013 - learning_rate: 0.0100
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7182 - loss: 0.6565 - val_accuracy: 0.7727 - val_loss: 0.5795 - learning_rate: 0.0100
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7480 - loss: 0.6217 - val_accuracy: 0.6818 - val_loss: 0.5750 - learning_rate: 0.0100
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7824 - loss: 0.5609 - val_accuracy: 0.6970 - val

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Reshape your preprocessed categorical data to add a channel dimension.
# If train_prepared has shape (num_samples, num_features), we reshape to (num_samples, num_features, 1)
train_prepared_reshaped = np.expand_dims(train_prepared, axis=-1)
test_prepared_reshaped = np.expand_dims(test_prepared, axis=-1)

# Define input shape for the model based on the reshaped data.
input_shape = train_prepared_reshaped.shape[1:]  # (num_features, 1)

# Build the CNN model using 1D convolutions
model = Sequential([
    Conv1D(filters=16, kernel_size=2, padding="same", activation="relu", input_shape=input_shape),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=32, kernel_size=2, padding="same", activation="relu"),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=64, kernel_size=2, padding="same", activation="relu"),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(500, activation="relu"),
    Dense(1, activation="sigmoid")  # Binary classification output
])

# Compile the model using binary crossentropy (assuming 'person_responses' is binary)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

# Train the model using the reshaped categorical data
history = model.fit(
    train_prepared_reshaped, train_labels,
    validation_data=(test_prepared_reshaped, test_labels),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, lr_scheduler]
)

test_loss, test_acc = model.evaluate(test_prepared_reshaped, test_labels)
print(f"Test Accuracy: {test_acc:.2f}")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.6279 - loss: 0.6575 - val_accuracy: 0.7273 - val_loss: 0.5789 - learning_rate: 0.0010
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7154 - loss: 0.6018 - val_accuracy: 0.7273 - val_loss: 0.5752 - learning_rate: 0.0010
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6981 - loss: 0.6050 - val_accuracy: 0.7273 - val_loss: 0.5723 - learning_rate: 0.0010
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6977 - loss: 0.5991 - val_accuracy: 0.7273 - val_loss: 0.5640 - learning_rate: 0.0010
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7097 - loss: 0.5738 - val_accuracy: 0.7273 - val_loss: 0.5623 - learning_rate: 0.0010
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6689 - loss: 0.6174 - val_accuracy: 0.7273 - val_loss: 0.5550 - learning_rate: 0.0010
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7171 - loss: 0.5676 - val_accuracy: 0.7273 - val_loss:

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

train_prepared_reshaped = np.expand_dims(train_prepared, axis=-1)
test_prepared_reshaped = np.expand_dims(test_prepared, axis=-1)
input_shape = train_prepared_reshaped.shape[1:]

def cnn_model_two(hp):
    model = Sequential()
    # 1st layer, using 1d cnn bc less data
    filters_1 = hp.Int('filters_1', min_value=16, max_value=64, step=16)
    kernel_size_1 = hp.Choice('kernel_size_1', values=[2, 3, 5])
    model.add(Conv1D(filters=filters_1,
                     kernel_size=kernel_size_1,
                     padding="same",
                     activation="relu",
                     input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # 2nd layer
    filters_2 = hp.Int('filters_2', min_value=32, max_value=128, step=32)
    kernel_size_2 = hp.Choice('kernel_size_2', values=[2, 3, 5])
    model.add(Conv1D(filters=filters_2,
                     kernel_size=kernel_size_2,
                     padding="same",
                     activation="relu"))
    model.add(MaxPooling1D(pool_size=2))

    # 3rd layer
    filters_3 = hp.Int('filters_3', min_value=32, max_value=128, step=32)
    kernel_size_3 = hp.Choice('kernel_size_3', values=[2, 3, 5])
    model.add(Conv1D(filters=filters_3,
                     kernel_size=kernel_size_3,
                     padding="same",
                     activation="relu"))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    # dense layer with tunable units - prepping for hyperparam tuning
    dense_units = hp.Int('dense_units', min_value=128, max_value=1024, step=128)
    model.add(Dense(dense_units, activation='relu'))


    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

early_stop = EarlyStopping(monitor="test_loss", patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="test_loss", factor=0.5, patience=3, verbose=1)

# tuner for keras is utilized and implementing random search
tuner = kt.RandomSearch(
    cnn_model_two,
    objective='test_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='hyper_tuning_dir',
    project_name='cnn_tuning_two'
)

tuner.search(
    train_prepared_reshaped, train_labels,
    epochs=50,
    batch_size=32,
    validation_data=(test_prepared_reshaped, test_labels),
    callbacks=[early_stop, lr_scheduler]
)

model_best_performance = tuner.get_best_models(num_models=1)[0]
test_loss, test_acc = model_best_performance.evaluate(test_prepared_reshaped, test_labels)
print(f"best test accuracy metric: {test_acc:.2f}")

Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 01m 13s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8153 - loss: 0.4697 
Best Test Accuracy: 0.82


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

# Reshaping input data for Conv1D
train_prepared_reshaped = np.expand_dims(train_prepared, axis=-1)
test_prepared_reshaped = np.expand_dims(test_prepared, axis=-1)
input_shape = train_prepared_reshaped.shape[1:]

def cnn_model_three(hp):
    model = Sequential()

    first_run_filters = hp.Int('first_run_filters', min_value=16, max_value=64, step=16) #first layer
    first_run_kernel_size = hp.Choice('first_run_kernel_size', values=[3, 5, 7])
    model.add(Conv1D(filters=first_run_filters, kernel_size=first_run_kernel_size, activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    second_run_filters = hp.Int('second_run_filters', min_value=32, max_value=128, step=32) #second
    second_run_kernel_size = hp.Choice('second_run_kernel_size', values=[3, 5, 7])
    model.add(Conv1D(filters=second_run_filters, kernel_size=second_run_kernel_size, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))

    third_run_filters = hp.Int('third_run_filters', min_value=32, max_value=128, step=32) #third layer
    third_run_kernel_size = hp.Choice('third_run_kernel_size', values=[3, 5, 7])
    model.add(Conv1D(filters=third_run_filters, kernel_size=third_run_kernel_size, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten()) #flatten
    dense_units = hp.Int('dense_units', min_value=128, max_value=512, step=128)
    model.add(Dense(dense_units, activation='relu'))

    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1) #dropout regularization
    model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid')) #output layer

    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# adjusting learning rates and stopping hyperparam tuning
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

#hyperparam tuning
tuner = kt.RandomSearch(
    cnn_model_three,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='hyper_tuning_dir',
    project_name='cnn_tuning_three'
)


# searching for the best hyperparameters
tuner.search(train_prepared_reshaped, train_labels, epochs=50, batch_size=32,
             validation_data=(test_prepared_reshaped, test_labels),
             callbacks=[early_stopping_callback, lr_scheduler_callback])

best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(test_prepared_reshaped, test_labels)
print(f"Best test accuracy: {test_accuracy:.2f}")

Trial 20 Complete [00h 00m 06s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7878788113594055
Total elapsed time: 00h 02m 53s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7885 - loss: 0.5631 
Best test accuracy: 0.79


In [ ]:
# also checked different feature columns to check for better accuracy
# the following were tested
# columns_to_keep = [ 'category', 'families', 'couples', 'solo', 'child-friendly', '21+', 'cost', 'interactivity', 'atmosphere', 'estimated_duration', 'person_responses']
# columns_to_keep = [ 'category', 'cost', 'interactivity', 'atmosphere', 'estimated_duration', 'person_responses']
